# Extraction

In [38]:
#Importing dependancies
import pandas as pd
from sqlalchemy import create_engine
from config import gkey
import requests
import json
from sqlalchemy.orm import Session

In [2]:
#Importing first table
toppings_path ="PizzaWeek.csv"
toppings_df = pd.read_csv(toppings_path)
toppings_df.rename(columns={"Topping": "toppings",
                           "Total": "total",
                           "Male":"male",
                           "Female":"female"}, inplace=True)
toppings_df.rename(columns={'total': 'num_total',
                           'male': 'num_male',
                           'female': 'num_female'}, inplace=True)
toppings_df

,toppings,num_total,num_male,num_female
0,Mushrooms,65,63,68
1,Onion,62,62,63
2,Ham,61,66,56
3,Peppers,60,63,57
4,Chicken,56,60,52
5,Pepperoni,56,66,46
6,Tomato,51,49,54
7,Bacon,49,56,43
8,Pineapple,42,40,44
9,Sweetcorn,42,38,46


In [3]:
#Renaming toppings df to match PostGres. 
toppings_df = toppings_df.rename(columns={
    'num_male':'male_total',
    'num_female':'female_total',
    'num_total':'grand_total'
})

# Re-ordering columns.
toppings_df = toppings_df[['toppings',
                           'male_total',
                           'female_total',
                           'grand_total']].copy()

toppings_df

,toppings,male_total,female_total,grand_total
0,Mushrooms,63,68,65
1,Onion,62,63,62
2,Ham,66,56,61
3,Peppers,63,57,60
4,Chicken,60,52,56
5,Pepperoni,66,46,56
6,Tomato,49,54,51
7,Bacon,56,43,49
8,Pineapple,40,44,42
9,Sweetcorn,38,46,42


In [4]:
#Reviewing count of each topping/verifying data types
toppings_df.count()

toppings        19
male_total      19
female_total    19
grand_total     19
dtype: int64

In [9]:
#Importing second table table
restaurants_path="restaurants.csv"
restaurants_df = pd.read_csv(restaurants_path)
restaurants_df.rename(columns= {"menuPageURL":"menupageurl",
                                "postalCode":"postalcode",
                                "priceRangeCurrency":"pricerangecurrency",
                                "priceRangeMin": "pricerangemin",
                                "priceRangeMax": "pricerangemax",
                                "menus.amountMax": "menus_amountmax",
                               "menus.amountMin" : "menus_amountmin",
                               "menus.currency": "menus_currency",
                               "menus.dateSeen": "menus_dateseen",
                               "menus.description": "menus_description",
                               "menus.name": "menus_name"}, inplace=True)
restaurants_df

,id,address,categories,city,country,keys,latitude,longitude,menupageurl,menus_amountmax,...,menus_currency,menus_dateseen,menus_description,menus_name,name,postalcode,pricerangecurrency,pricerangemin,pricerangemax,province
0,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Pizza Place,Bend,US,us/or/bend/cascadevillagemallacrossfromtarget/...,44.102665,-121.300797,NaN,22.50,...,USD,2016-03-31T02:19:42Z,NaN,Bianca Pizza,Little Pizza Paradise,97701,NaN,NaN,NaN,OR
1,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Pizza Place,Bend,US,us/or/bend/cascadevillagemallacrossfromtarget/...,44.102665,-121.300797,NaN,18.95,...,USD,2016-03-31T02:19:42Z,NaN,Cheese Pizza,Little Pizza Paradise,97701,NaN,NaN,NaN,OR
2,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,12.00,...,USD,2015-10-23T03:57:26Z,NaN,"Pizza, Margherita",The Brentwood,90049,USD,50.0,55.0,Brentwood
3,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,13.00,...,USD,2015-10-23T03:57:26Z,NaN,"Pizza, Mushroom",The Brentwood,90049,USD,50.0,55.0,Brentwood
4,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,13.00,...,USD,2015-10-23T03:57:26Z,"Olives, onions, capers, tomatoes","Pizza, Puttenesca",The Brentwood,90049,USD,50.0,55.0,Brentwood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3505,AVweYUZ0IN2L1WUf4Z9q,305 Ash St,Bar and Gastropub,Jefferson City,US,us/honeycreek/jeffersoncity/305ashst/-735513695,38.568717,-92.161596,NaN,11.99,...,USD,2016-07-06T20:05:49Z,NaN,Supreme Pizza,Prison Brews Brewery & Restaurant,65101,USD,25.0,40.0,Honey Creek
3506,AVweYUZ0IN2L1WUf4Z9q,305 Ash St,Bar and Gastropub,Jefferson City,US,us/honeycreek/jeffersoncity/305ashst/-735513695,38.568717,-92.161596,NaN,9.99,...,USD,2016-07-06T20:05:49Z,NaN,Vegetarian Pizza,Prison Brews Brewery & Restaurant,65101,USD,25.0,40.0,Honey Creek
3507,AVweZ5SbIN2L1WUf4nWU,4140 Carlisle Rd,"Restaurant,Italian Restaurant",Dover,US,us/davidsburg/dover/4140carlislerd/-1821521454,39.996444,-76.845180,NaN,5.00,...,USD,2015-10-23T01:03:47Z,NaN,Pita Pizza,Moonlight Cafe,17315,USD,0.0,30.0,Davidsburg
3508,AVweZ5SbIN2L1WUf4nWU,4140 Carlisle Rd,"Restaurant,Italian Restaurant",Dover,US,us/davidsburg/dover/4140carlislerd/-1821521454,39.996444,-76.845180,NaN,20.00,...,USD,2015-10-23T01:03:47Z,NaN,Steak Pizzaiola,Moonlight Cafe,17315,USD,0.0,30.0,Davidsburg


In [10]:
#Reviewing count of each topping/verifying data types
restaurants_df.count()

id                    3510
address               3510
categories            3510
city                  3510
country               3510
keys                  3510
latitude              3510
longitude             3510
menupageurl            610
menus_amountmax       2948
menus_amountmin       2948
menus_currency        2951
menus_dateseen        3510
menus_description     1526
menus_name            3510
name                  3510
postalcode            3484
pricerangecurrency    1557
pricerangemin         1557
pricerangemax         1557
province              3510
dtype: int64

In [11]:
#Removing unnecessary columns 
clean_restaurants_df = restaurants_df.drop(columns=['id',
                                                    'keys',
                                                    'menupageurl',
                                                    'menus_currency',
                                                    'menus_dateseen', 
                                                    'menus_description', 
                                                    'pricerangecurrency'])
clean_restaurants_df.rename(columns={'address': 'restaurant_address',
                                    'categories': 'restaurant_type',
                                    'menus_amountmax': 'max_price',
                                    'menus_amountmin': 'min_price',
                                    'menus_currency': 'currency',
                                    'menus_name': 'item_name',
                                    'name': 'restaurant_name',
                                    'postalcode': 'postal_code',
                                    'pricerangemin': 'price_range_min',
                                    'pricerangemax': 'price_range_max',
                                    }, inplace= True)

#Populating null fields with "0"
clean_restaurants_df['max_price'].fillna('0', inplace=True)
clean_restaurants_df['min_price'].fillna('0', inplace=True)
clean_restaurants_df['price_range_min'].fillna('0', inplace=True)
clean_restaurants_df['price_range_max'].fillna('0', inplace=True)
clean_restaurants_df.dropna(inplace=True)
clean_restaurants_df.drop_duplicates()
clean_restaurants_df

,restaurant_address,restaurant_type,city,country,latitude,longitude,max_price,min_price,item_name,restaurant_name,postal_code,price_range_min,price_range_max,province
0,Cascade Village Mall Across From Target,Pizza Place,Bend,US,44.102665,-121.300797,22.5,15.5,Bianca Pizza,Little Pizza Paradise,97701,0,0,OR
1,Cascade Village Mall Across From Target,Pizza Place,Bend,US,44.102665,-121.300797,18.95,18.95,Cheese Pizza,Little Pizza Paradise,97701,0,0,OR
2,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,34.064563,-118.469017,12,12,"Pizza, Margherita",The Brentwood,90049,50,55,Brentwood
3,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,34.064563,-118.469017,13,13,"Pizza, Mushroom",The Brentwood,90049,50,55,Brentwood
4,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,34.064563,-118.469017,13,13,"Pizza, Puttenesca",The Brentwood,90049,50,55,Brentwood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3505,305 Ash St,Bar and Gastropub,Jefferson City,US,38.568717,-92.161596,11.99,11.99,Supreme Pizza,Prison Brews Brewery & Restaurant,65101,25,40,Honey Creek
3506,305 Ash St,Bar and Gastropub,Jefferson City,US,38.568717,-92.161596,9.99,9.99,Vegetarian Pizza,Prison Brews Brewery & Restaurant,65101,25,40,Honey Creek
3507,4140 Carlisle Rd,"Restaurant,Italian Restaurant",Dover,US,39.996444,-76.845180,5,5,Pita Pizza,Moonlight Cafe,17315,0,30,Davidsburg
3508,4140 Carlisle Rd,"Restaurant,Italian Restaurant",Dover,US,39.996444,-76.845180,20,20,Steak Pizzaiola,Moonlight Cafe,17315,0,30,Davidsburg


# Transform

In [22]:
restaurant_chains_df = pd.DataFrame(clean_restaurants_df['restaurant_name'].value_counts())
restaurant_chains_df.rename(columns={'restaurant_name': 'count'}, inplace=True)
restaurant_chains_df.index.name = 'restaurant_name'
restaurant_chains_df.reset_index(inplace=True)
# restaurant_chains_df.index.name = 'chain_id'
# restaurant_chains_df.reset_index(inplace=True)
restaurant_chains_df

,restaurant_name,count
0,Sicilia's Pizzeria,96
1,J & G Restaurant,55
2,Casey's General Store,43
3,The Pizza Joint,36
4,North End Pizzeria,34
...,...,...
922,Brio Tuscan Grille - North Bethesda,1
923,Mazatlan Restaurant,1
924,Candlelight Dinner Playhouse,1
925,Brio Tuscan Grille - Marlton - The Promenade A...,1


In [16]:
#how many of each restaurant
unique_restaurants_df = pd.DataFrame(clean_restaurants_df['restaurant_name'].value_counts())
unique_restaurants_df.rename(columns={'restaurant_name': 'count'}, inplace=True)
unique_restaurants_df.index.name = 'restaurant_name'
unique_restaurants_df.reset_index(inplace=True)
unique_restaurants_df.index.name = 'chain_id'
unique_restaurants_df.reset_index(inplace=True)
unique_restaurants_df.drop(['count'], axis=1, inplace=True)
unique_restaurants_df

,chain_id,restaurant_name
0,0,Sicilia's Pizzeria
1,1,J & G Restaurant
2,2,Casey's General Store
3,3,The Pizza Joint
4,4,North End Pizzeria
...,...,...
922,922,Brio Tuscan Grille - North Bethesda
923,923,Mazatlan Restaurant
924,924,Candlelight Dinner Playhouse
925,925,Brio Tuscan Grille - Marlton - The Promenade A...


In [45]:
#location of each restaurant
location_df = clean_restaurants_df[['restaurant_name', 'restaurant_address', 'latitude', 'longitude', 'city', 'province', 'postal_code','country']]
location_df.index.name = 'location_id'

location_df = location_df.drop_duplicates()
location_df = location_df.dropna()
location_df.reset_index(inplace=True)
location_df

#Selecting columns to load. 
location_df = location_df[['location_id','restaurant_name', 'restaurant_address', 'latitude', 'longitude', 'city', 'postal_code','country']].copy()
location_df

,location_id,restaurant_name,restaurant_address,latitude,longitude,city,postal_code,country
0,0,Little Pizza Paradise,Cascade Village Mall Across From Target,44.102665,-121.300797,Bend,97701,US
1,2,The Brentwood,148 S Barrington Ave,34.064563,-118.469017,Los Angeles,90049,US
2,7,Bravo Pizza Hollywood,5142 Hollywood Blvd,34.101742,-118.301973,Los Angeles,90027,US
3,13,Lucky's Pub,801 Saint Emanuel St,29.752479,-95.354164,Houston,77003,US
4,14,Roadhouse Cafe,478 South St,41.648278,-70.291345,Hyannis,2601,US
...,...,...,...,...,...,...,...,...
976,3498,Rick's Cabaret,3551 Lafayette Rd,39.817155,-86.228120,Indianapolis,46222,US
977,3499,Mighty Mick's Pub & Cafe,10727 Randolph Saint Crown Point In,41.422509,-87.237723,Crown Point,46307,US
978,3500,Prison Brews Brewery & Restaurant,305 Ash St,38.568717,-92.161596,Jefferson City,65101,US
979,3507,Moonlight Cafe,4140 Carlisle Rd,39.996444,-76.845180,Dover,17315,US


In [ ]:
restaurant_location_df = pd.merge(location_df, unique_restaurants_df, how='left', on='restaurant_name')
restaurant_location_df

In [ ]:
restaurant_location_df = restaurant_location_df[['location_id', 'chain_id', 'restaurant_address', 'city', 'country', 'latitude', 'longitude', 'postal_code', 'province']]
restaurant_location_df.rename(columns={'chain_id': 'restaurant_id'}, inplace=True)
restaurant_location_df

In [40]:
#price 
price_range_df = clean_restaurants_df[['restaurant_name', 'price_range_min', 'price_range_max']]
price_range_df = price_range_df.drop_duplicates()
price_range_df.index.name = 'location_id'
price_range_df.reset_index(inplace=True)
price_range_df.drop(columns=['restaurant_name'], axis=1, inplace=True)
price_range_df

,location_id,price_range_min,price_range_max
0,0,0,0
1,2,50,55
2,7,0,0
3,13,25,40
4,14,0,0
...,...,...,...
932,3498,0,0
933,3499,0,0
934,3500,25,40
935,3507,0,30


## Load

In [5]:
# Initiating connection.
rds_connection_string = "postgres:M0neycarlo@localhost:5432/pizza_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [39]:
session = Session(bind=engine)

In [18]:
# Reviewing table names.
engine.table_names()

['restaraunt_chains', 'restaurant_chains_df', 'toppings_df']

In [7]:
#Import toppings_df
toppings_df.to_sql(name='toppings_df', con=engine, if_exists='append', index=False)

In [8]:
#Verify import is successful.  
pd.read_sql_query('select * from toppings_df', con=engine)

,toppings,male_total,female_total,grand_total
0,Mushrooms,63,68,65
1,Onion,62,63,62
2,Ham,66,56,61
3,Peppers,63,57,60
4,Chicken,60,52,56
5,Pepperoni,66,46,56
6,Tomato,49,54,51
7,Bacon,56,43,49
8,Pineapple,40,44,42
9,Sweetcorn,38,46,42


In [23]:
#Import restaraunt_chains_df 
restaurant_chains_df.to_sql(name='restaurant_chains_df', con=engine, if_exists='append', index=False)

In [24]:
#Ensure import is successful 
pd.read_sql_query('select * from restaurant_chains_df', con=engine)

,restaurant_name,count
0,Sicilia's Pizzeria,96
1,J & G Restaurant,55
2,Casey's General Store,43
3,The Pizza Joint,36
4,North End Pizzeria,34
...,...,...
922,Brio Tuscan Grille - North Bethesda,1
923,Mazatlan Restaurant,1
924,Candlelight Dinner Playhouse,1
925,Brio Tuscan Grille - Marlton - The Promenade A...,1


In [46]:
#Import location_df
location_df.to_sql(name='location_df', con=engine, if_exists='append', index=False)

In [47]:
#Ensure import is successful 
pd.read_sql_query('select * from location_df', con=engine)

,location_id,restaurant_name,restaurant_address,latitude,longitude,city,postal_code,country
0,NaN,Little Pizza Paradise,Cascade Village Mall Across From Target,44.102665,-121.300797,Bend,97701,US
1,NaN,The Brentwood,148 S Barrington Ave,34.064563,-118.469017,Los Angeles,90049,US
2,NaN,Bravo Pizza Hollywood,5142 Hollywood Blvd,34.101742,-118.301973,Los Angeles,90027,US
3,NaN,Lucky's Pub,801 Saint Emanuel St,29.752479,-95.354164,Houston,77003,US
4,NaN,Roadhouse Cafe,478 South St,41.648278,-70.291345,Hyannis,2601,US
...,...,...,...,...,...,...,...,...
1957,3498.0,Rick's Cabaret,3551 Lafayette Rd,39.817155,-86.228120,Indianapolis,46222,US
1958,3499.0,Mighty Mick's Pub & Cafe,10727 Randolph Saint Crown Point In,41.422509,-87.237723,Crown Point,46307,US
1959,3500.0,Prison Brews Brewery & Restaurant,305 Ash St,38.568717,-92.161596,Jefferson City,65101,US
1960,3507.0,Moonlight Cafe,4140 Carlisle Rd,39.996444,-76.845180,Dover,17315,US


In [41]:
#Import price_range_df
price_range_df.to_sql(name='price_range_df', con=engine, if_exists='append', index=False)

In [42]:
#Ensure import is successful 
pd.read_sql_query('select * from price_range_df', con=engine)

,location_id,price_range_min,price_range_max
0,0,0,0
1,2,50,55
2,7,0,0
3,13,25,40
4,14,0,0
...,...,...,...
932,3498,0,0
933,3499,0,0
934,3500,25,40
935,3507,0,30


In [48]:
#Review finalized database
pd.read_sql_query('select * from final_pizza_database', con=engine)

,Location ID,Restaurant Name,Restaurant Address,Rest. Chain Count,Latitude,Longitude,City,Postal Code,Country
0,0,Little Pizza Paradise,Cascade Village Mall Across From Target,2,44.102665,-121.300797,Bend,97701,US
1,2,The Brentwood,148 S Barrington Ave,5,34.064563,-118.469017,Los Angeles,90049,US
2,7,Bravo Pizza Hollywood,5142 Hollywood Blvd,6,34.101742,-118.301973,Los Angeles,90027,US
3,13,Lucky's Pub,801 Saint Emanuel St,1,29.752479,-95.354164,Houston,77003,US
4,14,Roadhouse Cafe,478 South St,2,41.648278,-70.291345,Hyannis,2601,US
...,...,...,...,...,...,...,...,...,...
932,3498,Rick's Cabaret,3551 Lafayette Rd,1,39.817155,-86.228120,Indianapolis,46222,US
933,3499,Mighty Mick's Pub & Cafe,10727 Randolph Saint Crown Point In,1,41.422509,-87.237723,Crown Point,46307,US
934,3500,Prison Brews Brewery & Restaurant,305 Ash St,7,38.568717,-92.161596,Jefferson City,65101,US
935,3507,Moonlight Cafe,4140 Carlisle Rd,2,39.996444,-76.845180,Dover,17315,US
